In [ ]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## The properties of PDFs
Probability density functions (PDFs) are characterized by certain parameters, that give the probability as a function of some quantity. The normal distribution, for example, is characterized by a mean value $\mu$ and and a standard deviation $\sigma$. We can play with PDFs using Pyhton's scipy and its stats package. The "standarized" normal is always centerer at 0 and has standard deviation of 1:

$$f(x) = \frac{\exp(-x^2/2)}{\sqrt{2\pi}}$$

But we can always change the $\mu$ and $\sigma$ using the parameters *loc* and *scale*:

In [ ]:
from scipy.stats import norm,gamma
fig, ax = plt.subplots(1, 1)
x = np.linspace(norm.ppf(0.001),norm.ppf(0.999), 1000)
ax.plot(x, norm.pdf(x), 'b-', lw=3, alpha=0.6, label='norm pdf')
x = np.linspace(norm.ppf(0.001,loc=2.0,scale=1.5),norm.ppf(0.999,loc=2.0,scale=1.5), 1000)
ax.plot(x, norm.pdf(x,loc=2.0,scale=1.5), 'r-', lw=3, alpha=0.6, label='norm pdf')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'PDF')

We can obtain samples from the distribution:

In [ ]:
r = norm.rvs(size=1000)
x = np.linspace(norm.ppf(0.001),norm.ppf(0.999), 1000)
plt.plot(x, norm.pdf(x), 'b-', lw=3, alpha=0.6, label='norm pdf')
plt.hist(r, density=True, histtype='stepfilled', alpha=0.2)
plt.legend(loc='best', frameon=False)
plt.xlabel(r'$x$')
plt.ylabel(r'N')

The *cumulative distribution function* measures the cumulative probability of the distribution, namely, the accumulated probability up to certain value. It is useful, for example, to answer the question: what is the probability of obtaining a value that is less than certain $x_0$?:

In [ ]:
fig, ax = plt.subplots(1, 1)
x = np.linspace(norm.ppf(0.001),norm.ppf(0.999), 1000)
ax.plot(x, norm.cdf(x), 'b-', lw=3, alpha=0.6, label='norm pdf')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'CDF')

## The Poisson distribution

The Poisson distribution is another discrete distribution, it expresses the probability of a given number of events occurring in a fixed interval of time (or space, volume, etc.). One assumption made is that these events occur with a known average rate and independently of each other. An example is the number of electrons detected by a sensor in an electron microscope during a time interval, or the number of soldiers in the Prussian army killed accidentally by horse kicks [(see here)](http://en.wikipedia.org/wiki/Poisson_distribution).

The Poisson distribution is defined as:


$$ f(k; \mu)= \frac{\mu^k e^{-\mu}}{k!}, $$

where $k$ is the number of events, $\mu$ is a positive real number, and $e$ is Euler's number ($e = 2.71828 \ldots$).

In [ ]:
from scipy.stats import poisson
# generate samples for different values of mu
kpts=np.arange(0,25)
for mu, c in zip([1,2, 4, 6], sns.color_palette()[:4]):
    randomVariates = poisson.rvs(mu, size=1000)
    plt.hist(randomVariates, normed=True, color=c, alpha=0.2, bins=range(0,26), label='$M=' + np.str(mu) + '$')
    plt.plot(kpts, poisson.pmf(kpts, mu), '.', color=c)

plt.legend()
plt.title("Poisson Distribution")
plt.xlabel("Number of Events")
plt.ylabel("Normed Counts")
plt.savefig('Posson_dist.png')

### X-ray counts in a Chandra Observation
We now read an X-ray dataset from Chandra, and try to look at the distribution of events as a function of time. The dataset is the same as for the first exercise. We first need to read the event file, and then we look at the photons hotto

In [ ]:
from astropy.io import fits
hdul = fits.open('/Users/jmartine/teaching/isya2018/lecture1/data/18344/primary/acisf18344N001_evt2.fits')

Now let us investigate how many photons hit the detector at a particular pixel position, per unit time. Is it always the same number? Let us plot the distribution of these numbers. Let us also try to guess a Poisson distribution that matches the observed values:

In [ ]:
# Create a mask to select the desired pixel
mask = ((hdul[1].data['ccd_id']==7) 
        & (hdul[1].data['dety']>=3961.)
        & (hdul[1].data['dety']<3962.)
        & (hdul[1].data['detx']>=4519.)
        & (hdul[1].data['detx']<4920.))

# Get the times of arrival of the photos for that pixel
times = np.array(hdul[1].data['time'][mask])

# Split the time into equal intervals
t_intvls = np.linspace(np.min(times),np.max(times),300)

# Count how many photos were detected in each equal time interval
n_events = []
for i in range(len(t_intvls[0:-1])):
    n_events.append(len(times[(times>t_intvls[i]) & (times<t_intvls[i+1])]))

#Plot the histogram
plt.hist(n_events,alpha=0.4)
plt.title('No. of photons detected on pixel (3961,4919) per unit time')
plt.xlabel('No. of photons')
plt.ylabel('N')

Let's now try to see if we can match this with a Poisson distribution

In [ ]:
kpts=np.arange(0,40)
mu = 15
plt.hist(n_events,alpha=0.4,normed=True,bins=15)
plt.plot(kpts, poisson.pmf(kpts, mu), '.', color='r')
plt.title('No. of photons detected on pixel (3961,4919) per unit time')
plt.xlabel('No. of photons')
plt.ylabel('N')

We now look at the energy of each photon. What is the distribution of photon energies? Let's have a look. Can we adjust a normal distribution to the energies?

In [ ]:
mask = (hdul[1].data['ccd_id']==7)
plt.hist(np.log10(hdul[1].data['energy'][mask]),bins=15,alpha=0.4,normed=True)
x = np.linspace(norm.ppf(0.0001,loc=3.25,scale=0.15),norm.ppf(0.9999,loc=3.25,scale=0.15), 1000)
plt.plot(x, norm.pdf(x,loc=3.25,scale=0.15), 'b-', lw=3, alpha=0.6, label='norm pdf')
plt.title('Distribution of photon energies')
plt.xlabel('log Energy (eV)')

The next question is how many photons in a given energy range hit the detector per unit time, regardless of their position. Let us have a look.

In [ ]:
mask = ((hdul[1].data['ccd_id']==7) 
        & (hdul[1].data['energy']>=10**3.264)
        & (hdul[1].data['energy']<10**3.266))

times = np.array(hdul[1].data['time'][mask])

t_intvls = np.linspace(np.min(times),np.max(times),300)

n_events = []
for i in range(len(t_intvls[0:-1])):
    n_events.append(len(times[(times>t_intvls[i]) & (times<t_intvls[i+1])]))
    
plt.hist(n_events,alpha=0.4)
plt.title('No. of 3.26 keV photons detected per unit time')
#plt.hist(np.log10(hdul[1].data['energy'][mask]),bins=15,alpha=0.4)

### Joint, marginal, and conditional probabilities
We now illustrate the concept of the different types of probabilities with an example. Suppose you roll two fair dice. What is the probability of getting 3 in one die and 2 in the other? Or 1 and 4, or 6 and 6? Also, what is the probability of getting 3 in one of them regardless of what you get in the other? And finally, what is the probability of getting 3 in one of them, given that you got exactly 3 in the other one? Let us perform the experiment by rolling our digital dice many times. First let us roll the dice a few times

In [ ]:
repeat = True
while repeat:
    print("You rolled",np.random.randint(1,7))
    print("Do you want to roll again?")
    repeat = ("y" or "yes") in input().lower()

And now let us roll two dice 100000 times, and illustrate joint, marginal, and conditional probabilities.

In [ ]:
n_rolls = 100000
d1 = np.random.randint(1,7,n_rolls)
d2 = np.random.randint(1,7,n_rolls)

# Here we count how many rolls of each pair we have
freqs = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        for roll in list(zip(d1,d2)):
            if (roll == (i+1,j+1)): 
                freqs[i,j]+= 1
                
freqs = freqs.astype(int)
print(freqs)


print(freqs[2,2]/100000.)
print(sum(freqs[:,2])/100000.)
print(freqs[:,1])

#print(list(zip(d1,d2)))
    

Let us define $n_{i,j}$ as the number of times the die 1 ends in number $i$ and die 2 ends in number $j$. Then the **joint** probability that both dice end up in three is:

$$P(d_1=3,d_2=3) = \frac{n_{3,3}}{N}$$

where $N$ is the total number of joint rolls:

In [ ]:
P_33 = freqs[2,2]/n_rolls
print(P_33)

The **marginal** probability of die 1 ending up in three is:

$$P(d_1=3) = \frac{\sum_j n_{3,j}}{N}$$

In [ ]:
P_3j = sum(freqs[:,2])/n_rolls
print(P_3j)

Finally, the **conditional** distribution that first die 1 ends in 3 given the second ones ends in 2 is:

$$P(d_1=3|d_2=2) = \frac{n_{3,2}}{\sum_i n_{i,2}}$$



In [ ]:
P_3_given_2 = freqs[2,1]/sum(freqs[:,1])
print(P_3_given_2)

### Exercise:
Consider the photons arriving to CCD 7 in the Cassiopea A dataset.
* Using CIAO, download observation 12020 from the Chandra archive. This corresponds to the Cassiopeia A supernova remnant.
* Produce an RGB image like last time, and identify regions dominated by photons of different energies. 
* Using DS9, load and save a few regions of interest.
* What is the distribution of the logarithmic photon energies in the for Cas A? What kind of distribution do you think that is?
* Find the mean and the standard deviations of the distribution, and record them
* Now do the same, but now limit yourself to each of the regions of interest.
* In each case, compare the histograms of energy with normal distributions with μ corresponding to the mean and σ corresponding to the standard deviation. What is expected value of the energy in each case?
* What is the probability of detecting a photon in each of the three ands s,m, and h energy bands for each of the regions? Discuss this in terms of conditional probability distributions.
* What is the probability of detecting hard or soft X-ray photons coming from a white dwarf?
* What does this tell you about the physical processes taking place in the different regions?

For this exercise we can either read the evt2 file using astropy, or just do the different searches in CIAO, save the tables, and then read the tables here.

In [ ]:
# Load the evt2 file
hdul = fits.open('/Users/jmartine/teaching/isya2018/lecture2/data/12020/repro/acisf12020_repro_evt2.fits')

#maks for only CCD 7
mask = ((hdul[1].data['ccd_id']==7))
energies = hdul[1].data['energy'][mask]

# Plot the histogram of energies
plt.hist(np.log10(hdul[1].data['energy'][mask]),bins=15,alpha=0.4,normed=True)
plt.title('Distribution of photon energies')
plt.xlabel('log Energy (eV)')

# Find the mean and standard deviation
# YOUR CODE HERE

# plot a normal distribution mathing the data
loca = 
stda = 
x = np.linspace(norm.ppf(0.0001,loc=loca,scale=stda),norm.ppf(0.9999,loc=loca,scale=stda), 1000)
plt.plot(x, norm.pdf(x,loc=loca,scale=stda), 'b-', lw=3, alpha=0.6, label='norm pdf')


# Creat masks with the different ehergies
h_band_mask = 
m_band_mask = 
s_band_mask = 


#The propbability of detecting an h band photon:


#The propbability of detecting an h band photon:


#The propbability of detecting an s band photon:


In [ ]:
# Now using the Cumulative Density Function:


Now we the regions with tables generated with CIAO with dmlist.

In [ ]:
# Region 1:
from astropy.io import ascii
data = ascii.read("reg1_energies.dat")  

In [ ]:
energies = data['energy'].data
plt.hist(np.log10(energies),bins=10,alpha=0.4,normed=True)
plt.title('Distribution of photon energies for Region 1')
plt.xlabel('log Energy (eV)')

# Find the mean and standard deviation


# plot a normal distribution mathing the data


# Create energy masks


#The propbability of detecting an h band photon:


#The propbability of detecting an h band photon:

#The propbability of detecting an s band photon:


In [ ]:
# Region 2
data = ascii.read("reg2_energies.dat")  

In [ ]:
energies = data['energy'].data
plt.hist(np.log10(energies),bins=10,alpha=0.4,normed=True)
plt.title('Distribution of photon energies for Region 1')
plt.xlabel('log Energy (eV)')

# Find the mean and standard deviation

# plot a normal distribution mathing the data

# Create energy bands


#The propbability of detecting an h band photon:

#The propbability of detecting an h band photon:


#The propbability of detecting an s band photon:


## Derivation of the Bayes' Theorem
We start with the product rule of probabilities. We know that:
$$P(A|B) = \frac{P(A,B)}{P(B)}$$
And also:
$$P(B|A) = \frac{P(A,B)}{P(A)}$$
Rearranging and combining these two equations, we get:
$$P(A|B)P(B) = P(A|B)P(A)$$
Now we divide both sides by $P(B)$:
$$P(A|B) = \frac{P(A|B)P(A)}{P(B)}$$
This is know as the Bayes' rule